In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('fede'))
if module_path not in sys.path:
    sys.path.append(module_path)
# import pandas as pd
from client import Client
from supported_modles import Supported_modles
import utils
from fedavg import Fedavg
from sklearn.utils.class_weight import compute_sample_weight
import numpy as np
import random
from copy import deepcopy

In [2]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [3]:
selected_model = Supported_modles.NN_classifier

In [4]:
clients, test_x, test_y = utils.set_data(selected_model, True, False)

Creating node1.
Creating node2.
Creating node3.
Creating node4.
Creating node5.


In [5]:
f1_before = []
for client in clients:
    client.init_empty_model(0.01)
    client.train_model(epochs=50)
    try: 
        score = client.test_model_f1(y_test=test_y, X_test=test_x)
    except IndexError:
        score = 0
    f1_before.append(score)
    print(score)

0.5783403082576298
0.5844880677444187
0.5949258018190523
0.6219054763690922


In [8]:
fedavg = Fedavg("global", 0.1, selected_model)

Waitiing for a Connection..


In [12]:
###First learn model on clients:
round_weights = []
dataset_size = 0
applicable_models = []
first = True

for client in clients:
    client.init_empty_model(0.01)
    X_train, X_test, y_train, y_test = client.split_data(0.8)

    # X_train = client.x_chunks[0]
    # y_train = client.y_chunks[0]

    client.train_model(X_train,y_train,epochs=10)
    dataset_size += client.x.shape[0]
    round_weights.append(dataset_size)
    applicable_models.append(client.model)
    if first:
        fedavg.init_global_model(client.model)
        first = False
    print('.')

round_weights = np.array(round_weights) / dataset_size # calculate weight based on actual dataset size
fedavg.update_global_model(applicable_models, round_weights, selected_model)

.
.
.
.
.


In [13]:
score = fedavg.test_model_f1(y_test=test_y, X_test=test_x)
print(score)

0.31340421823252573


In [14]:
number_of_rounds = 9
epochs = 10
max_score = 0
optimal_model = None

for round in range(number_of_rounds):
    print(round, end=' ')

    applicable_clients = random.sample((clients), 5)#random.randint(1, 4))
    applicable_models = []
    applicable_name = []
    round_weights = []
    dataset_size = 0
    

    for client in applicable_clients:
        print(f'.', end='')

        X_train, X_test, y_train, y_test = client.split_data(0.9)

        # X_train = client.x_chunks[round + 1]
        # y_train = client.y_chunks[round + 1]

       
        dataset_size += X_train.shape[0]
        sample_weights = compute_sample_weight('balanced', y=y_train)

        client.load_global_model(fedavg.model)
        client.train_local_agent(X_train, y_train, epochs, sample_weights)
        round_weights.append(X_train.shape[0])
        applicable_models.append(client.model)


    round_weights = np.array(round_weights) / dataset_size
    fedavg.update_global_model(applicable_models, round_weights, selected_model)

    score = fedavg.test_model_f1(y_test=test_y, X_test=test_x)
    if score > max_score:
        print(score)
        max_score = score
        optimal_model = deepcopy(fedavg.model)

0 .....0.8061039118323846
1 .....2 .....3 .....4 .....5 .....6 .....7 .....8 .....

In [15]:
# x = test_x
# y_hat = optimal_model.predict(x)
# score = f1_score(test_y,y_hat)
score = fedavg.test_model_f1(y_test=test_y, X_test=test_x)
print(score)
fedavg.model = optimal_model
score = fedavg.test_model_f1(y_test=test_y, X_test=test_x)
print(score)

0.6933368658100412
0.8061039118323846


: 

In [13]:
i = 0
for client in clients:
    print(f'Difference {score-f1_before[i]}')
    i += 1

Difference 0.0874815980287359
Difference 0.6879741247342779
Difference 0.6876085096520868
Difference 0.786331022741092
Difference 0.786331022741092


## Centralized Zone

In [14]:
client1 = Client("node1","0.0.0.0", 5001, selected_model)
client1.x = clients[0].x
client1.init_empty_model(0.01)


for client in clients:
    client1.train_model(x=client.x,y=client.y,epochs=50)
    print(f'.')

Creating node1.
.
.
.
.
.


In [15]:
score = client1.test_model_f1(y_test=test_y, X_test=test_x)
print(score)

0.7945415125621731
